This code calculates the 14 vegetation indices (VIs) from the preprocessed (mosaicked) Sentinel-2 data. It reads the satellite data (four bands:2, 3, 4 and 8) for the entire mosaicked area, one date folder at a time. After performing the calculations, it then clips the calculated VI rasters to the three study provinces (which is input as a shape file) and saves the outputs to tiff files (one file for each vegetation index) inside a folder "Gr2Indices" within the output folder. The output tiffs are named as satellite_date[yyyymmdd]_index.tif (eg. Sentinel2_20240802_GNDVI.tif). An 8-bit copy of all the output tiffs are also created in a folder called "COG_tiffs_8bit" within the "Gr2Indices" folder. The naming convention of these files are as COG_satellite_date[yyyymmdd]_index.tif (eg. COG_Sentinel2_20240802_GNDVI.tif). All the calculations are done locally. Temporary files are created locally during the calculations, but they are deleted at the end of the code execution. The code is structured in such a way that it handles a group of VIs at one time and not the whole lot. This has been done so as to conveninently execute the code even with a normal computer without requiring high end computational facilities.

In [ ]:
import os
import sys
import subprocess

# This gives the name of the environment directory
print("Environment name:", os.path.basename(sys.prefix))

In [ ]:
# Install necessary packages, if needed

required_packages = ["zipfile", "glob", "rasterio", "geopandas", "datetime", "re", "shutil"]

for package in required_packages:
    try:
        __import__(package if package != "scikit-learn" else "sklearn")
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("All packages have been installed!")

In [ ]:
# Import necessary packages
import tempfile
import os
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import numpy as np
from rasterio.transform import AffineTransformer
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling
import tarfile
import geopandas as gpd
from rasterio.mask import mask
import shutil
from rasterio.shutil import copy as rio_copy
import glob

In [ ]:
# User defined input and output paths
source_folder = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\1 - Image Processing\Processed_Sentinel2_VI_TIFFs"

# Date string (i.e., sub-folder name)
date_str = "20250324"

# Read AOI shapefile (e.g., provincial borders)
aoi_path = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\1 - Image Processing\Raw Data\GIS Maps and Shapefiles\Three Provinces (Old)\StudyAreaA4I.shp"  

# List of band numbers to process
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']

In [ ]:
input_folder_path = os.path.join(source_folder, date_str)
print(input_folder_path)

output_dir_date = os.path.join(output_folder, date_str)
os.makedirs(output_dir_date, exist_ok=True)
print(output_dir_date)

# Temporary folder to extract zip files
temp_folder = tempfile.TemporaryDirectory()
print(temp_folder.name)

In [ ]:
# Read AOI shapefile
aoi_gdf = gpd.read_file(aoi_path)
aoi_geom = [feature["geometry"] for feature in aoi_gdf.__geo_interface__["features"]]

In [ ]:
# Dictionary to store file paths for the required bands only
band_paths = {
    'B02': None,  # Blue
    'B03': None,  # Green
    'B04': None,  # Red
    'B08': None   # NIR
}

In [ ]:
# Required bands for Sentinel-2 processing
required_bands = ['B02', 'B03', 'B04', 'B08']

# Dictionary to store file paths for the required bands only
band_paths = {band: None for band in required_bands}

# Locate the relevant band files in the input folder
for file in os.listdir(input_folder_path):
    if file.endswith(".TIF"):
        for band in required_bands:
            if f"_{band}.TIF" in file:
                band_paths[band] = os.path.join(input_folder_path, file)

# Check if all required bands are present
missing_bands = [band for band, path in band_paths.items() if path is None]
if missing_bands:
    raise RuntimeError(f"Missing required bands: {', '.join(missing_bands)}")

In [ ]:
# Create the output directory within the main input folder
output_dir = os.path.join(input_folder_path, 'Gr2Indices')
os.makedirs(output_dir, exist_ok=True)

# Create temporary output directory within the main input folder
temp_dir = os.path.join(output_dir, "temp_indices")
os.makedirs(temp_dir, exist_ok=True)

In [ ]:
# Paths to reflectance bands for Sentinel-2
blue_path = band_paths['B02']   # Blue
green_path = band_paths['B03']  # Green
red_path = band_paths['B04']    # Red
nir_path = band_paths['B08']    # NIR

In [ ]:
# Define output paths for selected vegetation indices from Sentinel-2 (B02, B03, B04, B08)
temp_ndvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_NDVI_full.tif")
temp_evi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_EVI_full.tif")
temp_savi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_SAVI_full.tif")
temp_gndvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_GNDVI_full.tif")
temp_ndwi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_NDWI_full.tif")
temp_cig_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_CIG_full.tif")
temp_dvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_DVI_full.tif")
temp_rvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_RVI_full.tif")
temp_tvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_TVI_full.tif")
temp_msavi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_MSAVI_full.tif")
temp_pri_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_PRI_full.tif")
temp_wdrvi_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_WDRVI_full.tif")
temp_vari_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_VARI_full.tif")
temp_ari_path = os.path.join(temp_dir, f"Sentinel2_{date_str}_ARI_full.tif")

In [ ]:
# 1. Calculating NDVI, EVI, SAVI, GNDVI

with rasterio.open(band_paths['B08']) as nir_src, \
     rasterio.open(band_paths['B04']) as red_src, \
     rasterio.open(band_paths['B03']) as green_src, \
     rasterio.open(band_paths['B02']) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-9999.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    with rasterio.open(temp_ndvi_path, 'w', **meta) as ndvi_dst, \
         rasterio.open(temp_evi_path, 'w', **meta) as evi_dst, \
         rasterio.open(temp_savi_path, 'w', **meta) as savi_dst, \
         rasterio.open(temp_gndvi_path, 'w', **meta) as gndvi_dst:

        # Ensure all nodata values are known
        nir_nodata = nir_src.nodata or 0
        red_nodata = red_src.nodata or 0
        green_nodata = green_src.nodata or 0
        blue_nodata = blue_src.nodata or 0
             
        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

           # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)
        
            with np.errstate(divide='ignore', invalid='ignore'):
                # NDVI: (NIR - Red) / (NIR + Red)
                ndvi = (nir_masked - red_masked) / (nir_masked + red_masked)
                ndvi = np.where(np.isfinite(ndvi) & (ndvi >= -1) & (ndvi <= 1), ndvi, NODATA_VALUE)
                
                # EVI: 2.5 * (NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1)
                evi = 2.5 * (nir_masked - red_masked) / (nir_masked + 6 * red_masked - 7.5 * blue_masked + 1)
                evi = np.where(np.isfinite(evi) & (evi >= -1) & (evi <= 1), evi, NODATA_VALUE)

                # SAVI: ((NIR - Red) * (1 + L)) / (NIR + Red + L), L=0.5
                L = 0.5
                savi_denom = nir + red + L
                savi = np.where(valid_mask & (savi_denom != 0), ((nir - red) * (1 + L)) / savi_denom, NODATA_VALUE)
                savi = np.where((savi >= -0.3) & (savi <= 1.0), savi, NODATA_VALUE)

                # GNDVI: (NIR - Green) / (NIR + Green)
                gndvi_denom = nir + green
                gndvi = np.where(valid_mask & (gndvi_denom != 0), (nir - green) / gndvi_denom, NODATA_VALUE)
                gndvi = np.where((gndvi >= -1.0) & (gndvi <= 1.0), gndvi, NODATA_VALUE)
           
            # Write each index to disk
            ndvi_dst.write(ndvi, 1, window=window)
            evi_dst.write(evi, 1, window=window)
            savi_dst.write(savi, 1, window=window)
            gndvi_dst.write(gndvi, 1, window=window)

print("1. Calculation of NDVI, EVI, SAVI, GNDVI complete.")


In [ ]:
# 1. Clip and save final outputs for NDVI, EVI, SAVI, GNDVI

for temp_path, index_name in [
    (temp_ndvi_path, "NDVI"),
    (temp_evi_path, "EVI"),
    (temp_savi_path, "SAVI"),
    (temp_gndvi_path, "GNDVI")
]:
    try:
        with rasterio.open(temp_path) as src:
            # Clip using AOI geometry
            clipped_data, clipped_transform = mask(src, aoi_geom, crop=True, nodata=NODATA_VALUE)
            clipped_meta = src.meta.copy()
            clipped_meta.update({
                "height": clipped_data.shape[1],
                "width": clipped_data.shape[2],
                "transform": clipped_transform,
                "nodata": NODATA_VALUE,
                "compress": "lzw"  # Apply compression
            })

            # Define final output path for each index
            final_output_path = os.path.join(output_dir, f"Sentinel2_{date_str}_{index_name}.TIF")

            # Save clipped data to final output path
            with rasterio.open(final_output_path, 'w', **clipped_meta) as dst:
                dst.write(clipped_data)

            print(f" {index_name} successfully clipped and saved to:\n   {final_output_path}")
    except Exception as e:
        print(f" Error processing {index_name}: {e}")

print("\n1. Clipping and saving of NDVI, EVI, SAVI, GNDVI complete.")

In [ ]:
# 2. Calculating NDWI, CIG, DVI, TVI

with rasterio.open(band_paths['B08']) as nir_src, \
     rasterio.open(band_paths['B04']) as red_src, \
     rasterio.open(band_paths['B03']) as green_src, \
     rasterio.open(band_paths['B02']) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-9999.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    with rasterio.open(temp_ndwi_path, 'w', **meta) as ndwi_dst, \
         rasterio.open(temp_cig_path, 'w', **meta) as cig_dst, \
         rasterio.open(temp_dvi_path, 'w', **meta) as dvi_dst, \
         rasterio.open(temp_tvi_path, 'w', **meta) as tvi_dst:

        # Ensure all nodata values are known
        nir_nodata = nir_src.nodata or 0
        red_nodata = red_src.nodata or 0
        green_nodata = green_src.nodata or 0
        blue_nodata = blue_src.nodata or 0

        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

           # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)
            
            with np.errstate(divide='ignore', invalid='ignore'):
                # NDWI (McFeeters): (Green - NIR) / (Green + NIR)
                ndwi_denom = green_masked + nir_masked
                ndwi = np.where(valid_mask & (ndwi_denom != 0), (green_masked - nir_masked) / ndwi_denom, NODATA_VALUE)
                ndwi = np.where((ndwi >= -1.0) & (ndwi <= 1.0), ndwi, NODATA_VALUE)

                # CIG (Green Chlorophyll Index): NIR / Green - 1
                cig_denom = green_masked
                cig = np.where(valid_mask & (cig_denom != 0), nir_masked / cig_denom - 1, NODATA_VALUE)
                cig = np.where((cig >= 0.0) & (cig <= 5.5), cig, NODATA_VALUE)

                # DVI (Difference Vegetation Index): NIR - Red
                dvi = np.where(valid_mask, nir_masked - red_masked, NODATA_VALUE)
                dvi = np.where((dvi >= -1.0) & (dvi <= 1.0), dvi, NODATA_VALUE)

                # Transformed Vegetation Index (TVI)
                ndvi_denom = nir_masked + red_masked
                ndvi = np.where(ndvi_denom != 0, (nir_masked - red_masked) / ndvi_denom, NODATA_VALUE)
                ndvi = np.where((ndvi >= -1.0) & (ndvi <= 1.0), ndvi, NODATA_VALUE)
                tvi_raw = np.sqrt(np.clip(ndvi + 0.5, a_min=0, a_max=None))
                tvi = np.where((tvi_raw > 0.0) & (tvi_raw <= 1.3), tvi_raw, NODATA_VALUE)


            # Replace np.nan with nodata_value
            ndwi = np.nan_to_num(ndwi, nan=NODATA_VALUE).astype(np.float32)
            cig = np.nan_to_num(cig, nan=NODATA_VALUE).astype(np.float32)
            dvi = np.nan_to_num(dvi, nan=NODATA_VALUE).astype(np.float32)
            tvi = np.nan_to_num(tvi, nan=NODATA_VALUE).astype(np.float32)
                        
            # Write each index to disk
            ndwi_dst.write(ndwi, 1, window=window)
            cig_dst.write(cig, 1, window=window)
            dvi_dst.write(dvi, 1, window=window)
            tvi_dst.write(tvi, 1, window=window)

print("2. Calculation of NDWI, CIG, DVI, TVI complete.")


In [ ]:
# 2. Clip and save final outputs for NDWI, CIG, DVI, TVI

for temp_path, index_name in [
    (temp_ndwi_path, "NDWI"),
    (temp_cig_path, "CIG"),
    (temp_dvi_path, "DVI"),
    (temp_tvi_path, "TVI")
]:
    try:
        with rasterio.open(temp_path) as src:
            # Clip using AOI geometry
            clipped_data, clipped_transform = mask(src, aoi_geom, crop=True, nodata=NODATA_VALUE)
            clipped_meta = src.meta.copy()
            clipped_meta.update({
                "height": clipped_data.shape[1],
                "width": clipped_data.shape[2],
                "transform": clipped_transform,
                "nodata": NODATA_VALUE,
                "compress": "lzw"  # Apply compression
            })

            # Define final output path for each index
            final_output_path = os.path.join(output_dir, f"Sentinel2_{date_str}_{index_name}.TIF")

            # Save clipped data to final output path
            with rasterio.open(final_output_path, 'w', **clipped_meta) as dst:
                dst.write(clipped_data)

            print(f" {index_name} successfully clipped and saved to:\n   {final_output_path}")
    except Exception as e:
        print(f" Error processing {index_name}: {e}")

print("\n2. Clipping and saving of NDWI, CIG, DVI, TVI complete.")

In [ ]:
# 3. Calculating RVI, MSAVI, PRI, WDRVI

with rasterio.open(band_paths['B08']) as nir_src, \
     rasterio.open(band_paths['B04']) as red_src, \
     rasterio.open(band_paths['B03']) as green_src, \
     rasterio.open(band_paths['B02']) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-9999.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    with rasterio.open(temp_rvi_path, 'w', **meta) as rvi_dst, \
         rasterio.open(temp_msavi_path, 'w', **meta) as msavi_dst, \
         rasterio.open(temp_pri_path, 'w', **meta) as pri_dst, \
         rasterio.open(temp_wdrvi_path, 'w', **meta) as wdrvi_dst:
         
        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)
            
            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

           # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):

                # Ratio Vegetation Index (RVI)
                rvi = np.where(red_masked != 0, nir_masked / red_masked, NODATA_VALUE)
                rvi = np.where((rvi >= 0.5) & (rvi <= 10.0), rvi, NODATA_VALUE)

                # Modified Soil-Adjusted Vegetation Index (MSAVI)
                msavi_inner = (2 * nir_masked + 1) ** 2 - 8 * (nir_masked - red_masked)
                msavi = ((2 * nir_masked + 1) - np.sqrt(np.clip(msavi_inner, a_min=0, a_max=None))) / 2
                msavi = np.where((msavi >= -1.0) & (msavi <= 1.0), msavi, NODATA_VALUE)

                # Photochemical Reflectance Index (PRI)
                pri_denom = green_masked + red_masked
                pri = np.where(pri_denom != 0, (green_masked - red_masked) / pri_denom, NODATA_VALUE)
                pri = np.where((pri >= -1.0) & (pri <= 1.0), pri, NODATA_VALUE)

                # Wide Dynamic Range Vegetation Index (WDRVI)
                wdrvi_denom = 0.1 * nir_masked + red_masked
                wdrvi = np.where(wdrvi_denom != 0, (0.1 * nir_masked - red_masked) / wdrvi_denom, NODATA_VALUE)
                wdrvi = np.where((wdrvi >= -1.0) & (wdrvi <= 1.0), wdrvi, NODATA_VALUE)

                            
            # Replace np.nan with NODATA_VALUE for additional indices
            rvi = np.nan_to_num(rvi, nan=NODATA_VALUE).astype(np.float32)
            msavi = np.nan_to_num(msavi, nan=NODATA_VALUE).astype(np.float32) 
            pri = np.nan_to_num(pri, nan=NODATA_VALUE).astype(np.float32) 
            wdrvi = np.nan_to_num(wdrvi, nan=NODATA_VALUE).astype(np.float32) 
                        
            
            # Write each index to disk
            rvi_dst.write(rvi, 1, window=window)
            msavi_dst.write(msavi, 1, window=window)
            pri_dst.write(pri, 1, window=window)
            wdrvi_dst.write(wdrvi, 1, window=window)
            
print("3. Calculation of RVI, MSAVI, PRI, WDRVI complete.")

In [ ]:
# 3. Clip and save final outputs for RVI, MSAVI, PRI, WDRVI
for temp_path, index_name in [
    (temp_rvi_path, "RVI"),
    (temp_msavi_path, "MSAVI"),
    (temp_pri_path, "PRI"),
    (temp_wdrvi_path, "WDRVI")
]:
    try:
        with rasterio.open(temp_path) as src:
            # Clip using AOI geometry
            clipped_data, clipped_transform = mask(src, aoi_geom, crop=True, nodata=NODATA_VALUE)
            clipped_meta = src.meta.copy()
            clipped_meta.update({
                "height": clipped_data.shape[1],
                "width": clipped_data.shape[2],
                "transform": clipped_transform,
                "nodata": NODATA_VALUE,
                "compress": "lzw"  # Apply compression
            })

            # Define final output path for each index
            final_output_path = os.path.join(output_dir, f"Sentinel2_{date_str}_{index_name}.TIF")

            # Save clipped data to final output path
            with rasterio.open(final_output_path, 'w', **clipped_meta) as dst:
                dst.write(clipped_data)

            print(f" {index_name} successfully clipped and saved to:\n   {final_output_path}")
    except Exception as e:
        print(f" Error processing {index_name}: {e}")

print("\n3.Clipping and saving of RVI, MSAVI, PRI, WDRVI complete.")

In [ ]:
# 4. Calculating VARI, ARI

with rasterio.open(band_paths['B08']) as nir_src, \
     rasterio.open(band_paths['B04']) as red_src, \
     rasterio.open(band_paths['B03']) as green_src, \
     rasterio.open(band_paths['B02']) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-9999.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    with rasterio.open(temp_vari_path, 'w', **meta) as vari_dst, \
         rasterio.open(temp_ari_path, 'w', **meta) as ari_dst:

        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

           # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):
                # Visible Atmospherically Resistant Index (VARI)
                vari_denom = green_masked + red_masked - blue_masked
                vari = np.where(vari_denom != 0, (green_masked - red_masked) / vari_denom, NODATA_VALUE)
                vari = np.where((vari >= -1.0) & (vari <= 1.0), vari, NODATA_VALUE)

                # Anthocyanin Reflectance Index (ARI)
                ari = np.where((green_masked != 0) & (red_masked != 0), (1 / green_masked) - (1 / red_masked), NODATA_VALUE)
                ari = np.where((ari >= -6.0) & (ari <= 6.0), ari, NODATA_VALUE)

            
            # Replace np.nan with NODATA_VALUE for additional indices
            vari = np.nan_to_num(vari, nan=NODATA_VALUE).astype(np.float32)
            ari = np.nan_to_num(ari, nan=NODATA_VALUE).astype(np.float32)
                       
            
            # Write each index to disk
            vari_dst.write(vari, 1, window=window)
            ari_dst.write(ari, 1, window=window)

print("4. Calculation of VARI and ARI complete.")


In [ ]:
# 4. Clip and save final outputs for VARI and ARI

for temp_path, index_name in [
    (temp_vari_path, "VARI"),
    (temp_ari_path, "ARI")
]:
    try:
        with rasterio.open(temp_path) as src:
            # Clip using AOI geometry
            clipped_data, clipped_transform = mask(src, aoi_geom, crop=True, nodata=NODATA_VALUE)
            clipped_meta = src.meta.copy()
            clipped_meta.update({
                "height": clipped_data.shape[1],
                "width": clipped_data.shape[2],
                "transform": clipped_transform,
                "nodata": NODATA_VALUE,
                "compress": "lzw"  # Apply compression
            })

            # Define final output path for each index
            final_output_path = os.path.join(output_dir, f"Sentinel2_{date_str}_{index_name}.TIF")

            # Save clipped data to final output path
            with rasterio.open(final_output_path, 'w', **clipped_meta) as dst:
                dst.write(clipped_data)

            print(f" {index_name} successfully clipped and saved to:\n   {final_output_path}")
    except Exception as e:
        print(f" Error processing {index_name}: {e}")

print("\n4. Clipping and saving of VARI and ARI complete.")

In [ ]:
# Clean up temporary folder
shutil.rmtree(temp_dir)
print(" Temporary files deleted.")

In [ ]:
# Converting GeoTiffs to Cloud Optimized GeoTIFFs (COGs) : Open source and widely used
input_folder = output_dir
output_folder = os.path.join(input_folder, "COG_tiffs_8bit")
os.makedirs(output_folder, exist_ok=True)

# Find all .TIF files
tiff_files = glob.glob(os.path.join(input_folder, "*.TIF"))

for tiff_path in tiff_files:
    filename = os.path.basename(tiff_path)
    output_filename = "COG_" + filename
    output_path = os.path.join(output_folder, output_filename)


    with rasterio.open(tiff_path) as src:
        data = src.read(1)  # Read first band
        profile = src.profile.copy()
        nodata_in = -9999.0
        nodata_out = 255

        # Mask NoData values
        if nodata_in is not None:
            data = np.where(data == nodata_in, np.nan, data)

        # Rescale to 0–254
        min_val = np.nanmin(data)
        max_val = np.nanmax(data)

        if max_val == min_val:
            scaled = np.zeros_like(data)
        else:
            scaled = 254 * (data - min_val) / (max_val - min_val)

        # Set nodata areas to 255
        scaled_data = np.where(np.isnan(scaled), nodata_out, scaled).astype(np.uint8)

        # Update profile for 8-bit COG
        profile.update({
            "driver": "COG",
            "dtype": "uint8",
            "count": 1,
            "compress": "DEFLATE",
            "blocksize": 512,
            "tiled": True,
            "BIGTIFF": "IF_SAFER",
            "nodata": nodata_out
        })

        # Write the output
        with rasterio.open(output_path, "w", **profile) as dst:
            dst.write(scaled_data, 1)

print(" All TIFFs converted to 8-bit COG format and saved to:", output_folder)